In [1]:
df_gisa = spark.read.format("csv").options(inferSchema=True,header=True).option("delimiter",",").load("/home/ds/dev/Abc1115.csv")

In [2]:
df_gisa.show(10)

+------+-----+---+---+----+---+----+-----+--------+--------+--------+
|std_no|email|kor|eng|math|sci|hist|total|mgr_code|acc_code|loc_code|
+------+-----+---+---+----+---+----+-----+--------+--------+--------+
|990001| addx| 17| 29|  16| 49|  43|  154|       C|       A|       C|
|990002| stch| 30|  9|  48| 25|  81|  193|       C|       A|       A|
|990003| gali| 93| 60|   6| 84|  36|  279|       A|       C|       A|
|990004| miat| 73| 22|  48| 24|  72|  239|       B|       C|       C|
|990005| oran| 33| 38|  59| 58|  28|  216|       C|       B|       A|
|990006| sj24| 27| 65|  54| 77|  33|  256|       B|       A|       A|
|990007| kor2| 33| 85|  98| 60|  66|  342|       A|       B|       A|
|990008| vsky| 41|  5|  99| 14|  67|  226|       C|       A|       C|
|990009| kimd| 74|  4|   1| 33|  53|  165|       B|       C|       A|
|990010| blue| 55| 84|  50| 89|  37|  315|       A|       B|       A|
+------+-----+---+---+----+---+----+-----+--------+--------+--------+
only showing top 10 

In [3]:
answer_form = [["student number",10000]]
df_answers = spark.createDataFrame(answer_form,["title","answer"])

In [4]:
df_answers.printSchema()

root
 |-- title: string (nullable = true)
 |-- answer: long (nullable = true)



In [5]:
import pyspark.sql.functions as F

In [7]:
q1_format = df_gisa.select("std_no","kor","eng","loc_code")

In [8]:
q1 = q1_format.withColumn("score",F.col("kor")+F.col("eng"))

In [9]:
q1.where("loc_code='B'").selectExpr("max(score) as max_score").show()

+---------+
|max_score|
+---------+
|      192|
+---------+



In [12]:
schema = df_answers.schema
newDF = spark.createDataFrame([["max_score",192]],schema)
df_answers_v1=df_answers.union(newDF)

In [13]:
df_answers_v1.show()

+--------------+------+
|         title|answer|
+--------------+------+
|student number| 10000|
|     max_score|   192|
+--------------+------+



In [15]:
ans2 = q1.where("loc_code='B'").select("std_no", F.expr("score as _score")).orderBy(F.desc("_score"),"std_no" ).limit(10)

In [16]:
ans2.show()

+------+------+
|std_no|_score|
+------+------+
|990949|   192|
|990422|   190|
|990938|   187|
|990491|   184|
|990490|   180|
|990066|   175|
|990710|   172|
|990852|   172|
|990595|   169|
|990907|   166|
+------+------+



In [18]:
ans2.take(5)[4]._score

180

In [34]:
newDF = spark.createDataFrame([["5th_score",180]],schema)
df_answers_v2=df_answers_v1.union(newDF)

In [35]:
df_answers_v2.show()

+--------------+------+
|         title|answer|
+--------------+------+
|student number| 10000|
|     max_score|   192|
|     5th_score|   180|
+--------------+------+



In [25]:
q3_form = df_gisa.select("std_no","eng","math","acc_code","loc_code","total")

In [26]:
q3 = q3_form.withColumn("score",q3_form.eng+q3_form.math)

In [27]:
q3_1 = q3.where("score>=120")

In [28]:
q3_1.show(5)

+------+---+----+--------+--------+-----+-----+
|std_no|eng|math|acc_code|loc_code|total|score|
+------+---+----+--------+--------+-----+-----+
|990007| 85|  98|       B|       A|  342|  183|
|990010| 84|  50|       B|       A|  315|  134|
|990011| 69|  68|       A|       C|  295|  137|
|990013| 85|  36|       A|       C|  339|  121|
|990014| 89|  67|       C|       A|  393|  156|
+------+---+----+--------+--------+-----+-----+
only showing top 5 rows



In [31]:
q3_2 = q3_1.withColumn("rule",q3_1.total+F.when(q3_1.acc_code=='A',5).when(q3_1.acc_code=='B',15).otherwise(20))

In [32]:
q3_2.selectExpr("sum(rule)").show()

+---------+
|sum(rule)|
+---------+
|    99549|
+---------+



In [36]:
newDF = spark.createDataFrame([["accumulation",99549]],schema)
df_answers_v3=df_answers_v2.union(newDF)

In [37]:
df_answers_v3.show()

+--------------+------+
|         title|answer|
+--------------+------+
|student number| 10000|
|     max_score|   192|
|     5th_score|   180|
|  accumulation| 99549|
+--------------+------+



In [38]:
q4_form = df_gisa.where("acc_code != 'C'").select("std_no","kor","loc_code")

In [39]:
q4_1 = q4_form.withColumn("point",q4_form.kor+F.when(q4_form.loc_code=='A',5).when(q4_form.loc_code=='B',10).otherwise(15))

In [40]:
q4_1.where("point>=50").count()

397

In [41]:
newDF = spark.createDataFrame([["count",397]],schema)
df_answers_v4=df_answers_v3.union(newDF)

In [43]:
df_answers = df_answers_v4.where("answer!=10000")

In [44]:
df_answers.show()

+------------+------+
|       title|answer|
+------------+------+
|   max_score|   192|
|   5th_score|   180|
|accumulation| 99549|
|       count|   397|
+------------+------+



In [45]:
df_answers.write.format("jdbc") \
  .option("url", "jdbc:mysql://localhost:3306/bigdata") \
  .option("dbtable", "df_answers") \
  .option("user", "hive") \
  .option("password", "hive1234") \
  .save()